In [3]:
import os
from typing import Literal
from tavily import TavilyClient
from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model


tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
model = init_chat_model("openai:gpt-5-mini")


# Web search tool
def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a web search"""
    return tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )


# System prompt to steer the agent to be an expert researcher
research_instructions = """You are an expert researcher. Your job is to conduct thorough research, and then write a polished report.

You have access to an internet search tool as your primary means of gathering information.

## `internet_search`

Use this to run an internet search for a given query. You can specify the max number of results to return, the topic, and whether raw content should be included.
"""

# Create the deep agent
agent = create_deep_agent(
    model=model,
    tools=[internet_search],
    system_prompt=research_instructions,
)

# Invoke the agent
result = agent.invoke({"messages": [{"role": "user", "content": "What is langgraph?"}]})

In [6]:
print(result.keys())

dict_keys(['messages'])


In [13]:
print(result["messages"][-1].content)

Short answer
- LangGraph is an open-source, Python-based orchestration framework (built by the LangChain team) for creating stateful, long-running, graph-based AI workflows and agents. It treats an agent/workflow as a graph of nodes and edges that pass and update a shared state, enabling complex control flow, loops, streaming, human-in-the-loop, durable execution, and production deployment.

Key ideas and features
- StateGraph: workflows are explicit graphs where each node is a computation/step and the graph maintains a typed state object that flows between nodes.  
- Nodes & edges: nodes run code or call LLMs/tools; edges (including conditional edges) route execution to the next node.  
- Stateful & long-running: supports persistent state, checkpoints, and “time travel” / replay for debugging.  
- Orchestration primitives: streaming intermediate outputs, parallelization, breakpoints, human-in-the-loop actions.  
- Memory & stores: built-in patterns for short- and long-term memory and 